In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("GBP_CNY.csv")

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
df.head()

In [ ]:
df = df.loc[::-1]
df

In [ ]:
y_pred = df["Price"][:-2]
y_pred

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['month_year'] = df['date'].dt.to_period('M')
df

In [ ]:
month_year_future = df[df.year == 2022]
month_year_future.rename(columns={' value':'rmb'}, inplace=True)
month_year_future = month_year_future.groupby('month_year').rmb.mean().reset_index()["month_year"][3:]
month_year_future

In [ ]:
a =  df[df.year>2018]
a = a[df.year<2023]
a.rename(columns={' value':'rmb'}, inplace=True)
a = a.groupby('month_year').rmb.mean().reset_index()
a

In [ ]:
y_pred =  df[df.year>2018]
y_pred = y_pred[df.year<2023]
y_pred.rename(columns={' value':'rmb'}, inplace=True)
y_pred = y_pred.iloc[:-61]
y_pred

In [ ]:
df_groupby = y_pred.groupby('month_year').rmb.mean().reset_index()
df_groupby

In [ ]:
from statsmodels.tsa.stattools import adfuller
adf_test = adfuller(y_pred)
print('stat=%.3f, p=%.3f' % adf_test[0:2])
if adf_test[1] > 0.05:
 print('Probably not Stationary')
else:
 print('Probably Stationary')

In [ ]:
date = list(range(1,len(y_pred)+1))
date_fx = pd.DataFrame(zip(date, y_pred), columns=['Date', 'FX'])
date_fx_log = np.log(date_fx)
date_fx_log_diff = date_fx_log - date_fx_log.shift(1)
date_fx_log_diff.dropna(inplace=True)
y_stationary = date_fx_log_diff.iloc[:,1]

In [ ]:
adf_test_2 = adfuller(y_stationary)
print('stat=%.3f, p=%.3f' % adf_test_2[0:2])
if adf_test_2[1] > 0.05:
 print('Probably not Stationary')
else:
 print('Probably Stationary')

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
date_fx_log_diff = date_fx_log_diff.values.reshape(-1)
plot_acf(date_fx_log_diff, lags=50)
plot_pacf(date_fx_log_diff, lags=50)
plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
y = date_fx_log.iloc[:, 1]

In [ ]:
import itertools
p = range(1, 4)
d = range(2, 3)
q = range(1, 3)
pdq = list(itertools.product(p, d, q))
aics = []
params = []
for param in pdq:
    model = ARIMA(y, order=param)
    model_fit = model.fit()
    aic = model_fit.aic
    aics.append(aic)
    params.append(param)
combo = list(zip(aics, params))
combo.sort()
combo_array = np.array(combo)
print(combo_array)

In [ ]:
model_arima = ARIMA(y, (1,2,1))
model_arima_fit = model_arima.fit()
print(model_arima_fit.summary())

In [ ]:
pred = model_arima_fit.forecast(2, alpha=0.05)[0]
print(pred)
model_arima_fit.plot_predict(2,42)
#plt.plot(a["month_year"].astype(str), a["rmb"], color='r')
plt.show()

In [ ]:
pred = np.exp(pred)
forecast = pred.tolist()
#fx_2022 = np.array(list(zip(month_year_future, forecast)))
print(forecast)

In [ ]:
month_year_future

In [ ]:
x_merge = ['2021-12', '2022-01']
y_merge = [df_groupby.iloc[-1,1], forecast[0]]
plt.figure(figsize=(14,6))
plt.plot(df_groupby["month_year"].astype(str), df_groupby["rmb"], color='black')
plt.plot(a["month_year"].astype(str), a["rmb"], color='r')
plt.plot(x_merge, y_merge, color='y')
plt.plot(month_year_future.astype(str), forecast, color='y')
plt.xlabel("Month-Year")
plt.ylabel("Exchange Rate")
plt.xticks(fontsize=5)
plt.show()